In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import plotly.express as px

# Daten einlesen
gboxdf = pd.read_csv('gboxdata092025.csv', sep=';')
weatherdf = pd.read_csv('openmeteodata.csv', sep=',', skiprows=3)

# Konvertierung der Zeitstempel in datetime-Objekte
gboxdf['Timestamp'] = pd.to_datetime(gboxdf['Timestamp'])

In [ ]:
# Überblick über die Daten
print("Überblick über gboxdf:")
print(gboxdf.head())
print("\nInformationen über gboxdf:")
print(gboxdf.info())
print("\nStatistiken zu gboxdf:")
print(gboxdf.describe())

In [ ]:
# Korrigierte Filterung nach SensorID und MeasureName
# Beachte die Klammern um die Bedingungen
sensor_101_df = gboxdf[(gboxdf['SensorID'] == 101) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', 'Value']]
sensor_102_df = gboxdf[(gboxdf['SensorID'] == 102) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', 'Value']]
sensor_103_df = gboxdf[(gboxdf['SensorID'] == 103) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', 'Value']]
sensor_104_df = gboxdf[(gboxdf['SensorID'] == 104) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', 'Value']]
sensor_105_df = gboxdf[(gboxdf['SensorID'] == 105) & (gboxdf['MeasureName'] == 'moisture')][['Timestamp', 'Value']]

# Anzeige der ersten Zeilen von sensor_101_df
print("Erste Zeilen von sensor_101_df:")
print(sensor_101_df.head())

In [ ]:
# Temperaturdaten filtern
temp_df = gboxdf[(gboxdf['SensorID'] == 101) & (gboxdf['MeasureName'] == 'temperature')][['Timestamp', 'Value']]

# Temperaturverlauf plotten
plt.figure(figsize=(12, 6))
plt.plot(temp_df['Timestamp'], temp_df['Value'])
plt.title('Temperaturverlauf des Sensors 101')
plt.xlabel('Zeit')
plt.ylabel('Temperatur (°C)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Delta-Berechnung für Temperaturdaten
temp_delta_df = temp_df.copy()
temp_delta_df['Delta'] = temp_delta_df['Value'].diff()

# Ersten Wert entfernen, da hier kein Delta berechnet werden kann
temp_delta_df = temp_delta_df.dropna()

# Delta-Werte plotten
plt.figure(figsize=(12, 6))
plt.plot(temp_delta_df['Timestamp'], temp_delta_df['Delta'])
plt.title('Temperaturveränderungen des Sensors 101')
plt.xlabel('Zeit')
plt.ylabel('Temperaturänderung (°C)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Interaktiven Plot erstellen
fig = px.line(temp_df, x='Timestamp', y='Value', title='Interaktiver Temperaturverlauf')
fig.update_layout(xaxis_title='Zeit', yaxis_title='Temperatur (°C)')
fig.show()

In [ ]:
# Wetterdaten vorbereiten
weatherdf["date"] = pd.to_datetime(weatherdf["date"])

# Tagesbasierte Aggregation der Sensordaten für den Vergleich
temp_daily = temp_df.set_index('Timestamp').resample('D').mean().reset_index()
temp_daily['date'] = temp_daily['Timestamp'].dt.date

# Zusammenführen der Daten auf Tagesbasis
merged_data = pd.merge(temp_daily, weatherdf, on='date', how='inner')

# Korrelation zwischen Sensortemperatur und Wetterdaten plotten
if 'temperature_2m_mean' in weatherdf.columns:
    plt.figure(figsize=(10, 6))
    plt.scatter(merged_data['temperature_2m_mean'], merged_data['Value'])
    plt.title('Korrelation: Sensortemperatur vs. Wettertemperatur')
    plt.xlabel('Wettertemperatur (°C)')
    plt.ylabel('Sensortemperatur (°C)')
    plt.grid(True)
    plt.show()

In [ ]:
# Feuchtigkeitsdaten aller Sensoren visualisieren
plt.figure(figsize=(14, 8))
plt.plot(sensor_101_df['Timestamp'], sensor_101_df['Value'], label='Sensor 101')
plt.plot(sensor_102_df['Timestamp'], sensor_102_df['Value'], label='Sensor 102')
plt.plot(sensor_103_df['Timestamp'], sensor_103_df['Value'], label='Sensor 103')
plt.plot(sensor_104_df['Timestamp'], sensor_104_df['Value'], label='Sensor 104')
plt.plot(sensor_105_df['Timestamp'], sensor_105_df['Value'], label='Sensor 105')
plt.title('Feuchtigkeitsverlauf aller Sensoren')
plt.xlabel('Zeit')
plt.ylabel('Feuchtigkeit (%)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show

In [ ]:
# Statistische Analyse der Feuchtigkeit
moisture_stats = pd.DataFrame({
    'Sensor 101': sensor_101_df['Value'].describe(),
    'Sensor 102': sensor_102_df['Value'].describe(),
    'Sensor 103': sensor_103_df['Value'].describe(),
    'Sensor 104': sensor_104_df['Value'].describe(),
    'Sensor 105': sensor_105_df['Value'].describe()
})

print("Statistische Analyse der Feuchtigkeitswerte:")
print(moisture_stats)

# Boxplot für Feuchtigkeitsverteilung
plt.figure(figsize=(12, 6))
boxplot_data = [
    sensor_101_df['Value'],
    sensor_102_df['Value'],
    sensor_103_df['Value'],
    sensor_104_df['Value'],
    sensor_105_df['Value']
]
plt.boxplot(boxplot_data, labels=['Sensor 101', 'Sensor 102', 'Sensor 103', 'Sensor 104', 'Sensor 105'])
plt.title('Verteilung der Feuchtigkeitswerte nach Sensor')
plt.ylabel('Feuchtigkeit (%)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show

In [ ]:
# Tageszeitliche Analyse der Temperatur
temp_df['Hour'] = temp_df['Timestamp'].dt.hour
hourly_temp = temp_df.groupby('Hour')['Value'].mean()

plt.figure(figsize=(10, 6))
plt.bar(hourly_temp.index, hourly_temp.values)
plt.title('Durchschnittliche Temperatur nach Tageszeit')
plt.xlabel('Stunde des Tages')
plt.ylabel('Durchschnittstemperatur (°C)')
plt.xticks(range(0, 24))
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.show

In [ ]:
# Korrelationsanalyse mit Wetterdaten
if 'precipitation_sum' in weatherdf.columns and 'temperature_2m_mean' in weatherdf.columns:
    correlation_matrix = merged_data[['Value', 'temperature_2m_mean', 'precipitation_sum']].corr()

    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Korrelationsmatrix: Sensortemperatur und Wetterdaten')
    plt.tight_layout()
    plt.show()

    # Niederschlag und Feuchtigkeit
    moisture_daily = sensor_101_df.set_index('Timestamp').resample('D').mean().reset_index()
    moisture_daily['date'] = moisture_daily['Timestamp'].dt.date
    moisture_weather = pd.merge(moisture_daily, weatherdf, on='date', how='inner')

    plt.figure(figsize=(12, 6))
    fig, ax1 = plt.subplots(figsize=(12, 6))

    color = 'tab:blue'
    ax1.set_xlabel('Datum')
    ax1.set_ylabel('Feuchtigkeit (%)', color=color)
    ax1.plot(moisture_weather['date'], moisture_weather['Value'], color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Niederschlag (mm)', color=color)
    ax2.bar(moisture_weather['date'], moisture_weather['precipitation_sum'], color=color, alpha=0.5)
    ax2.tick_params(axis='y', labelcolor=color)

    plt.title('Feuchtigkeit und Niederschlag im Vergleich')
    fig.tight_layout()
    plt.xticks(rotation=45)
    plt.show()